In [ ]:
"""Library Inventory Management"""

import json
from pathlib import Path
import sys
from typing import List, Optional


class Book:
    def __init__(self, title: str, author: str, isbn: str, status: str = "available"):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.status = status  # "available" or "issued"

    def __str__(self) -> str:
        return f"{self.title} by {self.author} (ISBN: {self.isbn}) - {self.status}"

    def to_dict(self) -> dict:
        return {
            "title": self.title,
            "author": self.author,
            "isbn": self.isbn,
            "status": self.status,
        }

    def issue(self) -> None:
        if self.status == "issued":
            raise ValueError("Book is already issued.")
        self.status = "issued"

    def return_book(self) -> None:
        if self.status == "available":
            raise ValueError("Book is already available.")
        self.status = "available"

    def is_available(self) -> bool:
        return self.status == "available"


class LibraryInventory:
    def __init__(self, storage_path):
        self.storage_path = Path(storage_path)
        self.books: List[Book] = []
        self._load()

    def _load(self) -> None:
        try:
            if not self.storage_path.exists():
                self.books = []
                return
            text = self.storage_path.read_text(encoding="utf-8").strip()
            if not text:
                self.books = []
                return
            data = json.loads(text)
            built: List[Book] = []
            for item in data:
                if not isinstance(item, dict):
                    continue
                title = item.get("title", "")
                author = item.get("author", "")
                isbn = item.get("isbn", "")
                status = item.get("status", "available")
                if not isbn:
                    continue
                built.append(Book(title=title, author=author, isbn=isbn, status=status))
            self.books = built
        except Exception:
            # On any problem (missing/corrupted/parse error), start empty
            self.books = []

    def _save(self) -> None:
        try:
            self.storage_path.parent.mkdir(parents=True, exist_ok=True)
            data = [b.to_dict() for b in self.books]
            tmp = self.storage_path.with_suffix(self.storage_path.suffix + ".tmp")
            tmp.write_text(json.dumps(data, indent=2), encoding="utf-8")
            tmp.replace(self.storage_path)
        except Exception:
            # ignore save errors for simplicity
            pass

    def add_book(self, title: str, author: str, isbn: str) -> Book:
        if self.search_by_isbn(isbn) is not None:
            raise ValueError("A book with this ISBN already exists.")
        book = Book(title=title, author=author, isbn=isbn)
        self.books.append(book)
        self._save()
        return book

    def search_by_title(self, title_query: str) -> List[Book]:
        q = (title_query or "").lower()
        return [b for b in self.books if q in b.title.lower()]

    def search_by_isbn(self, isbn: str) -> Optional[Book]:
        for b in self.books:
            if b.isbn == isbn:
                return b
        return None

    def display_all(self) -> List[Book]:
        return list(self.books)

    def issue_book(self, isbn: str) -> None:
        book = self.search_by_isbn(isbn)
        if not book:
            raise ValueError("Book not found.")
        book.issue()
        self._save()

    def return_book(self, isbn: str) -> None:
        book = self.search_by_isbn(isbn)
        if not book:
            raise ValueError("Book not found.")
        book.return_book()
        self._save()


def print_menu() -> None:
    print("\nLibrary Inventory Manager")
    print("1. Add Book")
    print("2. Issue Book")
    print("3. Return Book")
    print("4. View All Books")
    print("5. Search by Title")
    print("6. Search by ISBN")
    print("7. Exit")


def safe_input(prompt: str) -> str:
    try:
        return input(prompt).strip()
    except (KeyboardInterrupt, EOFError):
        print("\nExiting.")
        sys.exit(0)


def main() -> None:
    # default storage path: ./data/books.json
    storage = Path.cwd() / "data" / "books.json"
    inv = LibraryInventory(storage)

    while True:
        print_menu()
        choice = safe_input("Choose an option (1-7): ")
        if choice == "1":
            title = safe_input("Title: ")
            author = safe_input("Author: ")
            isbn = safe_input("ISBN: ")
            if not isbn:
                print("ISBN cannot be empty.")
                continue
            try:
                inv.add_book(title, author, isbn)
                print("Book added successfully.")
            except Exception as e:
                print("Error:", e)

        elif choice == "2":
            isbn = safe_input("ISBN to issue: ")
            if not isbn:
                print("ISBN cannot be empty.")
                continue
            try:
                inv.issue_book(isbn)
                print("Book issued.")
            except Exception as e:
                print("Error:", e)

        elif choice == "3":
            isbn = safe_input("ISBN to return: ")
            if not isbn:
                print("ISBN cannot be empty.")
                continue
            try:
                inv.return_book(isbn)
                print("Book returned.")
            except Exception as e:
                print("Error:", e)

        elif choice == "4":
            books = inv.display_all()
            if not books:
                print("No books in catalog.")
            else:
                for b in books:
                    print(b)

        elif choice == "5":
            q = safe_input("Enter title search query: ")
            results = inv.search_by_title(q)
            if not results:
                print("No matches.")
            else:
                for r in results:
                    print(r)

        elif choice == "6":
            isbn = safe_input("Enter ISBN: ")
            if not isbn:
                print("ISBN cannot be empty.")
                continue
            b = inv.search_by_isbn(isbn)
            if not b:
                print("Not found.")
            else:
                print(b)

        elif choice == "7":
            print("Goodbye.")
            break
        else:
            print("Invalid option. Try again.")


if __name__ == "__main__":
    main()



Library Inventory Manager
1. Add Book
2. Issue Book
3. Return Book
4. View All Books
5. Search by Title
6. Search by ISBN
7. Exit


Choose an option (1-7):  6
Enter ISBN:  ISBN


Not found.

Library Inventory Manager
1. Add Book
2. Issue Book
3. Return Book
4. View All Books
5. Search by Title
6. Search by ISBN
7. Exit


Choose an option (1-7):  3
